# Data documentos (muestra total)

En este notebook se obtiene caracteristicas unicas documentos electronicos tipo 33 previo a la union con la data de cada contribuyente. 

In [1]:
##Se importan packages necesarios
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
from pyspark.sql.types import StringType,TimestampType
import matplotlib.pyplot as plt
from pyspark.sql import functions as F

In [2]:
#inicio de sesion en spark
ss_name = 'Lectura de datos Dashboard'
wg_conn = "spark.kerberos.access.hadoopFileSystems"
db_conn = "abfs://data@datalakesii.dfs.core.windows.net/"

spark = SparkSession.builder \
      .appName(f"Ejecucion algoritmo {ss_name}")  \
      .config(wg_conn, db_conn) \
      .config("spark.executor.memory", "6g") \
      .config("spark.driver.memory", "12g")\
      .config("spark.executor.cores", "4") \
      .config("spark.executor.instances", "5") \
      .config("spark.driver.maxResultSize", "12g") \
      .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.debug.maxToStringFields", "2000")

Setting spark.hadoop.yarn.resourcemanager.principal to hvega.externo


### Documentos electronicos tipo 33 y caracteristicas

In [3]:
# Seleciconamos de la dte emiso, receptor, folio, monto total, hora y fecha de emision respectivas

dte=spark.sql("select dhdr_folio,dtdc_codigo,dhdr_fch_emis, dhdr_rut_emisor,dhdr_dv_emisor,dhdr_rut_recep,dhdr_dv_recep,dhdr_mnt_total,dhdr_iva,dhdr_tmst_firma from DWBGDATA.HEADER_DTE_CONSOLIDADA_ENC_SAS_ANALITICA where dtdc_codigo=33")

Hive Session ID = 0a9659eb-89f3-4f16-9197-7496aad694ec


In [4]:
# 2. Obtener la última fecha de emisión
ultima_fecha_emision = dte.agg(F.max("dhdr_fch_emis")).collect()[0][0]

# 2. Calcular la fecha de hace 14 días (2 semanas)
fecha_antes = F.date_sub(F.lit(ultima_fecha_emision), 14)

# 3. Filtrar el DataFrame para las últimas 2 semanas
dte = dte.filter(dte["dhdr_fch_emis"] >= fecha_antes)


In [17]:
dte.count()

16890157

## Agregar variables de promedio y desviacion estandar de montos e iva para emisor y receptor en el periodo estudiado

Se agrega el monto total de emision para el emisor respectivo y el monto total recibido para ese receptor, ambos calculos realizados en la ventana de tiempo correspondiente.

In [6]:
# Agregar estadísticas para emis_CONT_RUT
agg_emisor = dte.groupBy("dhdr_rut_emisor").agg(
    F.avg("dhdr_mnt_total").alias("avg_dhdr_mnt_total_emisor"),
    F.stddev("dhdr_mnt_total").alias("stddev_dhdr_mnt_total_emisor"),
    F.avg("dhdr_iva").alias("avg_dhdr_iva_emisor"),
    F.stddev("dhdr_iva").alias("stddev_dhdr_iva_emisor")
)

# Agregar estadísticas para recep_CONT_RUT
agg_receptor = dte.groupBy("dhdr_rut_recep").agg(
    F.avg("dhdr_mnt_total").alias("avg_dhdr_mnt_total_receptor"),
    F.stddev("dhdr_mnt_total").alias("stddev_dhdr_mnt_total_receptor"),
    F.avg("dhdr_iva").alias("avg_dhdr_iva_receptor"),
    F.stddev("dhdr_iva").alias("stddev_dhdr_iva_receptor")
)


In [7]:
# Unir las estadísticas agregadas para emis_CONT_RUT
dte= dte.join(agg_emisor, on="dhdr_rut_emisor", how="left")

# Unir las estadísticas agregadas para recep_CONT_RUT
dte = dte.join(agg_receptor, on="dhdr_rut_recep", how="left")


In [8]:
dte.columns

['dhdr_rut_recep',
 'dhdr_rut_emisor',
 'dhdr_folio',
 'dtdc_codigo',
 'dhdr_fch_emis',
 'dhdr_dv_emisor',
 'dhdr_dv_recep',
 'dhdr_mnt_total',
 'dhdr_iva',
 'dhdr_tmst_firma',
 'avg_dhdr_mnt_total_emisor',
 'stddev_dhdr_mnt_total_emisor',
 'avg_dhdr_iva_emisor',
 'stddev_dhdr_iva_emisor',
 'avg_dhdr_mnt_total_receptor',
 'stddev_dhdr_mnt_total_receptor',
 'avg_dhdr_iva_receptor',
 'stddev_dhdr_iva_receptor']

In [9]:
from pyspark.sql import functions as F

dte = dte.withColumn("anio", F.year("dhdr_tmst_firma")) \
         .withColumn("mes", F.month("dhdr_tmst_firma")) \
         .withColumn("dia", F.dayofmonth("dhdr_tmst_firma")) \
         .withColumn("hora", F.hour("dhdr_tmst_firma")) \
         .withColumn("es_fin_de_semana", 
             F.when(F.date_format("dhdr_tmst_firma", "u").cast("int").isin([6, 7]), 1).otherwise(0)) \
         .withColumn("bloque_horario", 
             F.when((F.col("hora") >= 0) & (F.col("hora") < 6), "Madrugada")
              .when((F.col("hora") >= 6) & (F.col("hora") < 12), "Mañana")
              .when((F.col("hora") >= 12) & (F.col("hora") < 19), "Tarde")
              .otherwise("Noche")) \
         .withColumn("dia_semana", F.dayofweek("dhdr_tmst_firma")) \
         .withColumn("semana_mes", 
             (F.dayofmonth("dhdr_tmst_firma") - 1) / 7 + 1)


## Se guarda dataset en datalake

In [10]:
# Cambiar la política de análisis de tiempo
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Se guarda el archivo final en el datalake. 
dte.write.mode('overwrite').format("parquet").save("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/APA/Analisis_factura/dtes_no_estratificado")


------------------------------------------------------------------------------------------------------------------------

## Cesión de documentos tributarios

## Completar

In [11]:
doct=spark.table("hivenom.csn_doctos_final")
ces=spark.table("hivenom.csn_cesion_final")

In [12]:
# Realiza el inner join
dte_cesion = dte.join(
    doct,  # Asumiendo que ya tienes el DataFrame 'ces'
    (dte.dhdr_folio == doct.rdoc_folio) &
    (dte.dhdr_rut_emisor == doct.rdoc_rut_emisor_e) &
    (dte.dhdr_dv_emisor == doct.rdoc_dv_emisor),
    "left"
).select(
    dte.dhdr_folio,
    dte.dtdc_codigo,
    dte.dhdr_rut_emisor,
    dte.dhdr_dv_emisor,
    dte.dhdr_rut_recep,
    dte.dhdr_dv_recep,
    dte.dhdr_mnt_total,
    dte.dhdr_iva,
    dte.dhdr_fch_emis,  # Asegúrate de que esta columna exista en dte
    *doct.columns # Selecciona todas las columnas de 'ces'
)

# Agrega la columna 'cedido' para indicar si hay cruce
dte_cesion = dte_cesion.withColumn(
    "cedido",
    F.when(doct.rdoc_folio.isNotNull(), "Sí").otherwise("No")
)



In [13]:
dte_cesion.columns

['dhdr_folio',
 'dtdc_codigo',
 'dhdr_rut_emisor',
 'dhdr_dv_emisor',
 'dhdr_rut_recep',
 'dhdr_dv_recep',
 'dhdr_mnt_total',
 'dhdr_iva',
 'dhdr_fch_emis',
 'rdoc_codigo',
 'rtdc_codigo',
 'rdoc_rut_emisor_e',
 'rdoc_dv_emisor',
 'rdoc_rut_receptor_e',
 'rdoc_dv_receptor',
 'rdoc_folio',
 'rdoc_fch_emis',
 'rdoc_fch_emis_int',
 'rdoc_fch_emis_str',
 'rdoc_mnt_total',
 'rdoc_codigo_sii',
 'rdoc_firma_dte',
 'rdoc_firma_doc',
 'rtes_estado',
 'rdoc_info_acuse',
 'rdoc_rut_tenedor_e',
 'rdoc_dv_tenedor',
 'cont_rut_emisor_e',
 'cont_dv_emisor',
 'cont_rut_receptor_e',
 'cont_dv_receptor',
 'cont_rut_tenedor_e',
 '_c29',
 'cedido']

In [14]:
# Agrupa por las columnas originales de dte y cuenta las veces que ha sido cedido
result = dte_cesion.groupBy(
    "dhdr_folio",
    "dtdc_codigo",
    "dhdr_fch_emis",
    "dhdr_rut_emisor",
    "dhdr_dv_emisor",
    "dhdr_rut_recep",
    "dhdr_dv_recep",
    "dhdr_mnt_total",
    "dhdr_iva"
).agg(
    F.count(F.when(F.col("cedido") == "Sí", 1)).alias("veces_cedido")  # Cuenta solo los cedidos
)

In [15]:
result.show()

ERROR:root:KeyboardInterrupt while sending command. 380][Stage 20:> (0 + 0) / 200]]]
Traceback (most recent call last):
  File "/home/cdsw/.local/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cdsw/.local/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Realiza el left join con dte_cesion
dte_cesion_det = dte_cesion.join(
    ces,  # Asumiendo que ya tienes el DataFrame 'ces'
    dte_cesion.rdoc_codigo == ces.rdoc_codigo,  # Asegúrate de que esta columna exista
    "left"  # Cambiado a left join
).select(
    dte_cesion.dhdr_folio,
    dte_cesion.dhdr_rut_emisor,
    dte_cesion.dhdr_rut_recep
)

# Muestra el resultado (opcional)
dte_cesion_det.show()

In [ ]:
result.show()

## Obtencion de documentos de RCV

In [ ]:
rcv=spark.table("DWBGDATA.DCV_GENERIC_DET_CONSOLIDADO_SAS")

## Cruce con registros de RCV

In [ ]:
# Realiza el left join
union= dte.join(
    rcv,
    (dte.dhdr_folio == rcv.det_folio_doc_ref) &
    (dte.dhdr_rut_emisor ==rcv.dcv_rut_emisor_e) &
    (dte.dhdr_rut_recep == rcv.det_rut_doc_e),
    "left"
)

# Agrega una columna para indicar si hubo cruce
union = union.withColumn(
    "cruce_rcv",
    F.when(rcv.det_folio_doc_ref.isNotNull(), "Sí").otherwise("No")
)


In [ ]:
# Selecciona todas las columnas originales de dte y las columnas específicas de rcv
df_final = union.select(
    dte.dhdr_folio,
    dte.dtdc_codigo,
    dte.dhdr_fch_emis,
    dte.dhdr_rut_emisor,
    dte.dhdr_dv_emisor,
    dte.dhdr_rut_recep,
    dte.dhdr_dv_recep,
    dte.dhdr_mnt_total,
    dte.dhdr_iva,
    rcv.det_emisor_nota,
    rcv.det_fch_doc,
    rcv.det_fec_creacion,
    rcv.tipo_transaccion,
    F.when(rcv.det_folio_doc_ref.isNotNull(), "Sí").otherwise("No").alias("cruce_rcv")
)

# Muestra el resultado
df_final.show()


### Cesion de documentos

In [ ]:
spark.stop()